In [42]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import math
from itertools import combinations
from datetime import datetime
from music21 import *

For the reward function:  
if correct segmentation: output +1 #(change of roughness)  
if incorrect segmentation: output -1#(correct change of roughness in the next segmentation)  
if correct do nothing : output +1  
if incorrect do nothing: output -1 #(correct change of roughness)  
if illegal: output 0  

In [58]:
class SegmentationEnv(Env):  # Fit one particular coin first
    def __init__(self, pieces):
        #Preprocess the pieces
        self.notes = []
        self.offset = []
        self.beat = []
        self.duration = []
        self.octave = []
        self.correct_offset = []
#         self.beatchanges = []
        for piece in pieces:
            xnotes = []
            xoffset = []
            xbeat = []
            xduration = []
            xoctave = []
            xcoroffset = []
            c = converter.parse(piece)
            post = c.flattenParts().flat
            for note in post.notes:
                duration = note.duration.quarterLength
                offset = note.offset
                beat = note.beat
                if note.lyric is not None and note.offset != 0:
                    xcoroffset.append(note.offset)
                allnotes = list(note.pitches)
                for note1 in allnotes:
                    xnotes.append(note1.name)
                    xoffset.append(offset)
                    xbeat.append(beat)
                    xduration.append(duration)
                    xoctave.append(note1.octave)
            self.notes.append(xnotes)
            self.offset.append(xoffset)
            self.beat.append(xbeat)
            self.duration.append(xduration)
            self.octave.append(xoctave)
            self.correct_offset.append(xcoroffset)
            #             xbeatchange = {}
#             for ts in post.recurse().getElementsByClass(meter.TimeSignature):
#                 assert ts.denominator in [2,4,8]
#                 if ts.denominator == 2:
#                     xbeatchange[ts.offset] = 2
#                 elif ts.denominator == 4:
#                     xbeatchange[ts.offset] = 1
#                 else:
#                     xbeatchange[ts.offset] = 0.5
#             self.beatchanges.append(xbeatchange)

        #Actions: Remain segment (0), segment (1)
        self.action_space = Discrete(2)
        
        #Observations: First dim 12 pitch classes, Second dim Octave (1-7), Value is total duration.
        self.observation_space = Box(
            low=np.zeros((12,7),dtype=np.float32),
            high=np.ones((12,7),dtype=np.float32)*20, # Set the maximum duration to 20. If exceed, then just keep at 20.
        )
        
        #internal state: check where the time currently is 
        self.current_piece = 0
        self.current_noteoffset = 0
        self.notelistfirst = 0
        self.notelistlast = 0
        self.latestbeatfirst = 0
        self.latestbeatlast = 0
        self.state = np.zeros((12,7))
        
        #save segmentation for rendering purposes
        self.determined_offset = []
        
    def step(self, action):
        #Calculating reward
        if action == 0: # do nothing
            is_segment = False
            if self.current_noteoffset not in self.correct_offset[self.current_piece]: #correct
                reward = 1
            else:
                reward = max(-self.change_in_roughness()/20,-1)
        else: # segmentation
            is_segment = True
            if self.current_noteoffset == 0: #illegal operations
                reward = 0
            else:
                self.determined_offset.append((self.current_piece,self.current_noteoffset))
                if self.current_noteoffset in self.correct_offset[self.current_piece]:
                    reward = 1
                else:
                    reward = -1    
        #determine new obs state
        if is_segment and self.current_noteoffset != 0:
            self.notelistfirst = self.latestbeatfirst
        done = False
        if self.latestbeatlast == len(self.beat[self.current_piece]): #Finished a piece
            self.current_piece += 1
            if self.current_piece == len(self.notes):
                done = True
            else:
                done = False
                self.current_noteoffset = 0
                self.notelistfirst = 0
                self.notelistlast = 0 
                self.latestbeatfirst = 0
                self.latestbeatlast = 0
        if not done:
            self.current_noteoffset = self.offset[self.current_piece][self.latestbeatlast]
            currentbeat = self.beat[self.current_piece][self.latestbeatlast]
            currentindex = self.latestbeatlast + 1
            self.latestbeatfirst = self.latestbeatlast
            while len(self.beat[self.current_piece]) > currentindex and self.beat[self.current_piece][currentindex]//1 == currentbeat:
                currentindex += 1
            self.notelistlast = currentindex
            self.latestbeatlast = currentindex
        info = {}
        return self.staterender(done), reward, done, info

    def render(self):
#         print("Current notelist:",self.notelistfirst,self.notelistlast)
        print("The current segmentation are:")
        for segment in self.determined_offset:
            print(segment)
        return
    
    def change_in_roughness(self):
        def roughness(notes):
            '''
            Calculate the Roughness of notes according to sum of ideal ratio N+M
            Reference: https://www.researchgate.net/publication/276905584_Measuring_Musical_Consonance_and_Dissonance
            '''
            def interval_to_ratio(interval):
                interval_ratio_mapping = {
                    0:1+1,
                    1:18+17,
                    2:9+8,
                    3:6+5,
                    4:5+4,
                    5:4+3,
                    6:17+12,
                    7:3+2,
                    8:8+5,
                    9:5+3,
                    10:16+9,
                    11:17+9,
                    12:2+1
                }
                interval_pitch_mapping = {
                    1:0,
                    2:2,
                    3:4,
                    4:5,
                    5:7,
                    6:9,
                    7:11,
                    8:12
                }
                ans = interval_pitch_mapping[int(interval[-1])]
                if int(interval[-1]) in [4,5,8]:
                    intname = interval[:-1]
                    if intname == "dd":
                        ans -= 2
                    elif intname == "d":
                        ans -= 1
                    elif intname == "A":
                        ans += 1
                    elif intname == "AA":
                        ans += 2
                else:
                    intname = interval[:-1]
                    if intname == "m":
                        ans -= 1
                    elif intname == "d":
                        ans -= 2
                    elif intname == "A":
                        ans += 1
                    elif intname == "AA":
                        ans += 2
                if ans < 0:
                    ans = ans%12
                return interval_ratio_mapping[ans]
            ans = 0
            for combo in combinations(notes,2):
                n1 = note.Note(combo[0])
                n2 = note.Note(combo[1])
                xinterval = interval.Interval(noteStart=n1,noteEnd=n2)
                ans += interval_to_ratio(xinterval.semiSimpleName)
            return ans/len(notes) if len(notes)!= 0 else 0
        notelist1 = []
        for i in range(self.notelistfirst,self.latestbeatfirst):
            notelist1.append(self.notes[self.current_piece][i])
        notelist2 = notelist1.copy()
        for i in range(self.latestbeatfirst,self.latestbeatlast):
            notelist2.append(self.notes[self.current_piece][i])
        return abs(roughness(notelist2)-roughness(notelist1))
    
    def staterender(self,done):
        pitch_to_index = {"C": 0, "D": 2, "E": 4, "F": 5, "G": 7, "A": 9, "B": 11}
        obsarray = np.zeros((12,7))
        notelist = []
        if done:
            return obsarray
        for idx in range(self.notelistfirst,self.notelistlast):
            current_note = self.notes[self.current_piece][idx]
            notelist.append(current_note)
            current_duration = self.duration[self.current_piece][idx]
            current_octave = self.octave[self.current_piece][idx]
            pitchindex = pitch_to_index[current_note[0]]
            current_note = current_note[1:]
            if current_note == "#":
                pitchindex += 1
            elif current_note == "##":
                pitchindex += 2
            elif current_note == "-":
                pitchindex -= 1
            elif current_note == "--":
                pitchindex -= 2
            pitchindex = pitchindex % 12
            if current_octave < 1 or current_octave > 7:
                continue
            current_octave -= 1
            obsarray[pitchindex][current_octave] += current_duration
            obsarray[pitchindex][current_octave] = min(20,obsarray[pitchindex][current_octave])
#         print(notelist)
        obsarray = obsarray/20
        return obsarray

    def reset(self):
        self.current_piece = 0
        self.current_noteoffset = 0
        self.notelistfirst = 0
        self.notelistlast = 0 #exclusive
        self.latestbeatfirst = 0
        self.latestbeatlast = 0 #exclusive
        currentbeat = self.beat[self.current_piece][self.latestbeatlast]
        currentindex = self.latestbeatlast + 1
        while len(self.beat[self.current_piece]) > currentindex and self.beat[self.current_piece][currentindex]//1 == currentbeat:
            currentindex += 1
        self.notelistlast = currentindex
        self.latestbeatlast = currentindex
        return self.staterender(False)

In [53]:
import time
env = SegmentationEnv(["../review/not1_Prelude_I.musicxml","../review/not_Arabesque_No._1_in_E_Major.musicxml"])
cur_state = env.reset()
done = False
while not done:
    env.render()
    action = env.action_space.sample()
    print("Action taken:",action)
    new_state, reward, done, _ = env.step(action)
    print("Reward:",reward)


The current segmentation are:
Action taken: 1
Reward: 0
The current segmentation are:
Action taken: 0
Reward: 1
The current segmentation are:
Action taken: 1
Reward: -1
The current segmentation are:
(0, 2.0)
Action taken: 1
Reward: -1
The current segmentation are:
(0, 2.0)
(0, 3.0)
Action taken: 1
Reward: 1
The current segmentation are:
(0, 2.0)
(0, 3.0)
(0, 4.0)
Action taken: 1
Reward: -1
The current segmentation are:
(0, 2.0)
(0, 3.0)
(0, 4.0)
(0, 5.0)
Action taken: 1
Reward: -1
The current segmentation are:
(0, 2.0)
(0, 3.0)
(0, 4.0)
(0, 5.0)
(0, 6.0)
Action taken: 0
Reward: 1
The current segmentation are:
(0, 2.0)
(0, 3.0)
(0, 4.0)
(0, 5.0)
(0, 6.0)
Action taken: 1
Reward: 1
The current segmentation are:
(0, 2.0)
(0, 3.0)
(0, 4.0)
(0, 5.0)
(0, 6.0)
(0, 8.0)
Action taken: 0
Reward: 1
The current segmentation are:
(0, 2.0)
(0, 3.0)
(0, 4.0)
(0, 5.0)
(0, 6.0)
(0, 8.0)
Action taken: 1
Reward: -1
The current segmentation are:
(0, 2.0)
(0, 3.0)
(0, 4.0)
(0, 5.0)
(0, 6.0)
(0, 8.0)
(0, 10.

(0, 48.0)
(0, 50.0)
(0, 52.0)
(0, 57.0)
(0, 58.0)
(0, 61.0)
(0, 62.0)
(0, 63.0)
(0, 65.0)
(0, 69.0)
Action taken: 1
Reward: -1
The current segmentation are:
(0, 2.0)
(0, 3.0)
(0, 4.0)
(0, 5.0)
(0, 6.0)
(0, 8.0)
(0, 10.0)
(0, 11.0)
(0, 12.0)
(0, 16.0)
(0, 17.0)
(0, 18.0)
(0, 19.0)
(0, 20.0)
(0, 22.0)
(0, 24.0)
(0, 26.0)
(0, 27.0)
(0, 28.0)
(0, 29.0)
(0, 30.0)
(0, 33.0)
(0, 34.0)
(0, 35.0)
(0, 36.0)
(0, 39.0)
(0, 40.0)
(0, 42.0)
(0, 44.0)
(0, 45.0)
(0, 48.0)
(0, 50.0)
(0, 52.0)
(0, 57.0)
(0, 58.0)
(0, 61.0)
(0, 62.0)
(0, 63.0)
(0, 65.0)
(0, 69.0)
(0, 74.0)
Action taken: 1
Reward: -1
The current segmentation are:
(0, 2.0)
(0, 3.0)
(0, 4.0)
(0, 5.0)
(0, 6.0)
(0, 8.0)
(0, 10.0)
(0, 11.0)
(0, 12.0)
(0, 16.0)
(0, 17.0)
(0, 18.0)
(0, 19.0)
(0, 20.0)
(0, 22.0)
(0, 24.0)
(0, 26.0)
(0, 27.0)
(0, 28.0)
(0, 29.0)
(0, 30.0)
(0, 33.0)
(0, 34.0)
(0, 35.0)
(0, 36.0)
(0, 39.0)
(0, 40.0)
(0, 42.0)
(0, 44.0)
(0, 45.0)
(0, 48.0)
(0, 50.0)
(0, 52.0)
(0, 57.0)
(0, 58.0)
(0, 61.0)
(0, 62.0)
(0, 63.0)
(0, 65.0

(0, 39.0)
(0, 40.0)
(0, 42.0)
(0, 44.0)
(0, 45.0)
(0, 48.0)
(0, 50.0)
(0, 52.0)
(0, 57.0)
(0, 58.0)
(0, 61.0)
(0, 62.0)
(0, 63.0)
(0, 65.0)
(0, 69.0)
(0, 74.0)
(0, 75.0)
(0, 77.0)
(0, 78.0)
(0, 79.0)
(0, 80.0)
(0, 82.0)
(0, 84.0)
(0, 85.0)
(0, 86.0)
(0, 87.0)
(0, 88.0)
(0, 90.0)
(0, 93.0)
(0, 97.0)
(0, 102.0)
(0, 103.0)
(0, 106.0)
(0, 107.0)
(0, 109.0)
Action taken: 0
Reward: 1
The current segmentation are:
(0, 2.0)
(0, 3.0)
(0, 4.0)
(0, 5.0)
(0, 6.0)
(0, 8.0)
(0, 10.0)
(0, 11.0)
(0, 12.0)
(0, 16.0)
(0, 17.0)
(0, 18.0)
(0, 19.0)
(0, 20.0)
(0, 22.0)
(0, 24.0)
(0, 26.0)
(0, 27.0)
(0, 28.0)
(0, 29.0)
(0, 30.0)
(0, 33.0)
(0, 34.0)
(0, 35.0)
(0, 36.0)
(0, 39.0)
(0, 40.0)
(0, 42.0)
(0, 44.0)
(0, 45.0)
(0, 48.0)
(0, 50.0)
(0, 52.0)
(0, 57.0)
(0, 58.0)
(0, 61.0)
(0, 62.0)
(0, 63.0)
(0, 65.0)
(0, 69.0)
(0, 74.0)
(0, 75.0)
(0, 77.0)
(0, 78.0)
(0, 79.0)
(0, 80.0)
(0, 82.0)
(0, 84.0)
(0, 85.0)
(0, 86.0)
(0, 87.0)
(0, 88.0)
(0, 90.0)
(0, 93.0)
(0, 97.0)
(0, 102.0)
(0, 103.0)
(0, 106.0)
(0, 107.0)
(

Reward: -1
The current segmentation are:
(0, 2.0)
(0, 3.0)
(0, 4.0)
(0, 5.0)
(0, 6.0)
(0, 8.0)
(0, 10.0)
(0, 11.0)
(0, 12.0)
(0, 16.0)
(0, 17.0)
(0, 18.0)
(0, 19.0)
(0, 20.0)
(0, 22.0)
(0, 24.0)
(0, 26.0)
(0, 27.0)
(0, 28.0)
(0, 29.0)
(0, 30.0)
(0, 33.0)
(0, 34.0)
(0, 35.0)
(0, 36.0)
(0, 39.0)
(0, 40.0)
(0, 42.0)
(0, 44.0)
(0, 45.0)
(0, 48.0)
(0, 50.0)
(0, 52.0)
(0, 57.0)
(0, 58.0)
(0, 61.0)
(0, 62.0)
(0, 63.0)
(0, 65.0)
(0, 69.0)
(0, 74.0)
(0, 75.0)
(0, 77.0)
(0, 78.0)
(0, 79.0)
(0, 80.0)
(0, 82.0)
(0, 84.0)
(0, 85.0)
(0, 86.0)
(0, 87.0)
(0, 88.0)
(0, 90.0)
(0, 93.0)
(0, 97.0)
(0, 102.0)
(0, 103.0)
(0, 106.0)
(0, 107.0)
(0, 109.0)
(0, 113.0)
(0, 114.0)
(0, 116.0)
(0, 117.0)
(0, 119.0)
(0, 121.0)
(0, 122.0)
(0, 124.0)
(0, 126.0)
(0, 129.0)
(0, 130.0)
(0, 131.0)
(1, 1.0)
(1, 3.0)
(1, 5.0)
(1, 6.0)
(1, 9.0)
Action taken: 0
Reward: 1
The current segmentation are:
(0, 2.0)
(0, 3.0)
(0, 4.0)
(0, 5.0)
(0, 6.0)
(0, 8.0)
(0, 10.0)
(0, 11.0)
(0, 12.0)
(0, 16.0)
(0, 17.0)
(0, 18.0)
(0, 19.0)
(0,

Reward: -0.747638248847926
The current segmentation are:
(0, 2.0)
(0, 3.0)
(0, 4.0)
(0, 5.0)
(0, 6.0)
(0, 8.0)
(0, 10.0)
(0, 11.0)
(0, 12.0)
(0, 16.0)
(0, 17.0)
(0, 18.0)
(0, 19.0)
(0, 20.0)
(0, 22.0)
(0, 24.0)
(0, 26.0)
(0, 27.0)
(0, 28.0)
(0, 29.0)
(0, 30.0)
(0, 33.0)
(0, 34.0)
(0, 35.0)
(0, 36.0)
(0, 39.0)
(0, 40.0)
(0, 42.0)
(0, 44.0)
(0, 45.0)
(0, 48.0)
(0, 50.0)
(0, 52.0)
(0, 57.0)
(0, 58.0)
(0, 61.0)
(0, 62.0)
(0, 63.0)
(0, 65.0)
(0, 69.0)
(0, 74.0)
(0, 75.0)
(0, 77.0)
(0, 78.0)
(0, 79.0)
(0, 80.0)
(0, 82.0)
(0, 84.0)
(0, 85.0)
(0, 86.0)
(0, 87.0)
(0, 88.0)
(0, 90.0)
(0, 93.0)
(0, 97.0)
(0, 102.0)
(0, 103.0)
(0, 106.0)
(0, 107.0)
(0, 109.0)
(0, 113.0)
(0, 114.0)
(0, 116.0)
(0, 117.0)
(0, 119.0)
(0, 121.0)
(0, 122.0)
(0, 124.0)
(0, 126.0)
(0, 129.0)
(0, 130.0)
(0, 131.0)
(1, 1.0)
(1, 3.0)
(1, 5.0)
(1, 6.0)
(1, 9.0)
(1, 12.0)
(1, 15.0)
(1, 16.0)
(1, 19.0)
(1, 20.0)
(1, 21.0)
Action taken: 0
Reward: 1
The current segmentation are:
(0, 2.0)
(0, 3.0)
(0, 4.0)
(0, 5.0)
(0, 6.0)
(0, 8.

...

(1, 305.0)
(1, 307.0)
(1, 308.0)
(1, 309.0)
(1, 310.0)
(1, 311.0)
(1, 313.0)
(1, 314.0)
(1, 316.0)
(1, 321.0)
(1, 323.0)
(1, 324.0)
(1, 327.0)
(1, 330.0)
(1, 331.0)
(1, 332.0)
(1, 333.0)
(1, 334.0)
(1, 335.0)
(1, 338.0)
(1, 339.0)
(1, 340.0)
(1, 341.0)
(1, 343.0)
(1, 347.0)
(1, 348.0)
(1, 349.0)
(1, 350.0)
(1, 351.0)
(1, 352.0)
(1, 353.0)
(1, 355.0)
(1, 358.0)
(1, 361.0)
(1, 363.0)
(1, 366.0)
(1, 367.0)
(1, 369.0)
(1, 371.0)
(1, 372.0)
(1, 373.0)
(1, 374.0)
(1, 375.0)
(1, 377.0)
(1, 378.0)
(1, 381.0)
(1, 383.0)
(1, 385.0)
(1, 386.0)
(1, 387.0)
(1, 388.0)
Action taken: 1
Reward: 1
The current segmentation are:
(0, 2.0)
(0, 3.0)
(0, 4.0)
(0, 5.0)
(0, 6.0)
(0, 8.0)
(0, 10.0)
(0, 11.0)
(0, 12.0)
(0, 16.0)
(0, 17.0)
(0, 18.0)
(0, 19.0)
(0, 20.0)
(0, 22.0)
(0, 24.0)
(0, 26.0)
(0, 27.0)
(0, 28.0)
(0, 29.0)
(0, 30.0)
(0, 33.0)
(0, 34.0)
(0, 35.0)
(0, 36.0)
(0, 39.0)
(0, 40.0)
(0, 42.0)
(0, 44.0)
(0, 45.0)
(0, 48.0)
(0, 50.0)
(0, 52.0)
(0, 57.0)
(0, 58.0)
(0, 61.0)
(0, 62.0)
(0, 63.0)
(0, 65.0)

(0, 84.0)
(0, 85.0)
(0, 86.0)
(0, 87.0)
(0, 88.0)
(0, 90.0)
(0, 93.0)
(0, 97.0)
(0, 102.0)
(0, 103.0)
(0, 106.0)
(0, 107.0)
(0, 109.0)
(0, 113.0)
(0, 114.0)
(0, 116.0)
(0, 117.0)
(0, 119.0)
(0, 121.0)
(0, 122.0)
(0, 124.0)
(0, 126.0)
(0, 129.0)
(0, 130.0)
(0, 131.0)
(1, 1.0)
(1, 3.0)
(1, 5.0)
(1, 6.0)
(1, 9.0)
(1, 12.0)
(1, 15.0)
(1, 16.0)
(1, 19.0)
(1, 20.0)
(1, 21.0)
(1, 30.0)
(1, 31.0)
(1, 38.0)
(1, 39.0)
(1, 41.0)
(1, 42.0)
(1, 43.0)
(1, 45.0)
(1, 46.0)
(1, 48.0)
(1, 49.0)
(1, 52.0)
(1, 53.0)
(1, 55.0)
(1, 58.0)
(1, 59.0)
(1, 61.0)
(1, 66.0)
(1, 67.0)
(1, 70.0)
(1, 72.0)
(1, 73.0)
(1, 74.0)
(1, 75.0)
(1, 77.0)
(1, 81.0)
(1, 84.0)
(1, 86.0)
(1, 87.0)
(1, 88.0)
(1, 89.0)
(1, 91.0)
(1, 92.0)
(1, 95.0)
(1, 96.0)
(1, 98.0)
(1, 100.0)
(1, 101.0)
(1, 102.0)
(1, 103.0)
(1, 105.0)
(1, 106.0)
(1, 107.0)
(1, 109.0)
(1, 110.0)
(1, 111.0)
(1, 112.0)
(1, 115.0)
(1, 116.0)
(1, 117.0)
(1, 122.0)
(1, 124.0)
(1, 125.0)
(1, 127.0)
(1, 130.0)
(1, 131.0)
(1, 132.0)
(1, 136.0)
(1, 137.0)
(1, 138.0)
(1, 

(0, 34.0)
(0, 35.0)
(0, 36.0)
(0, 39.0)
(0, 40.0)
(0, 42.0)
(0, 44.0)
(0, 45.0)
(0, 48.0)
(0, 50.0)
(0, 52.0)
(0, 57.0)
(0, 58.0)
(0, 61.0)
(0, 62.0)
(0, 63.0)
(0, 65.0)
(0, 69.0)
(0, 74.0)
(0, 75.0)
(0, 77.0)
(0, 78.0)
(0, 79.0)
(0, 80.0)
(0, 82.0)
(0, 84.0)
(0, 85.0)
(0, 86.0)
(0, 87.0)
(0, 88.0)
(0, 90.0)
(0, 93.0)
(0, 97.0)
(0, 102.0)
(0, 103.0)
(0, 106.0)
(0, 107.0)
(0, 109.0)
(0, 113.0)
(0, 114.0)
(0, 116.0)
(0, 117.0)
(0, 119.0)
(0, 121.0)
(0, 122.0)
(0, 124.0)
(0, 126.0)
(0, 129.0)
(0, 130.0)
(0, 131.0)
(1, 1.0)
(1, 3.0)
(1, 5.0)
(1, 6.0)
(1, 9.0)
(1, 12.0)
(1, 15.0)
(1, 16.0)
(1, 19.0)
(1, 20.0)
(1, 21.0)
(1, 30.0)
(1, 31.0)
(1, 38.0)
(1, 39.0)
(1, 41.0)
(1, 42.0)
(1, 43.0)
(1, 45.0)
(1, 46.0)
(1, 48.0)
(1, 49.0)
(1, 52.0)
(1, 53.0)
(1, 55.0)
(1, 58.0)
(1, 59.0)
(1, 61.0)
(1, 66.0)
(1, 67.0)
(1, 70.0)
(1, 72.0)
(1, 73.0)
(1, 74.0)
(1, 75.0)
(1, 77.0)
(1, 81.0)
(1, 84.0)
(1, 86.0)
(1, 87.0)
(1, 88.0)
(1, 89.0)
(1, 91.0)
(1, 92.0)
(1, 95.0)
(1, 96.0)
(1, 98.0)
(1, 100.0)
(1, 101

Reward: -1
The current segmentation are:
(0, 2.0)
(0, 3.0)
(0, 4.0)
(0, 5.0)
(0, 6.0)
(0, 8.0)
(0, 10.0)
(0, 11.0)
(0, 12.0)
(0, 16.0)
(0, 17.0)
(0, 18.0)
(0, 19.0)
(0, 20.0)
(0, 22.0)
(0, 24.0)
(0, 26.0)
(0, 27.0)
(0, 28.0)
(0, 29.0)
(0, 30.0)
(0, 33.0)
(0, 34.0)
(0, 35.0)
(0, 36.0)
(0, 39.0)
(0, 40.0)
(0, 42.0)
(0, 44.0)
(0, 45.0)
(0, 48.0)
(0, 50.0)
(0, 52.0)
(0, 57.0)
(0, 58.0)
(0, 61.0)
(0, 62.0)
(0, 63.0)
(0, 65.0)
(0, 69.0)
(0, 74.0)
(0, 75.0)
(0, 77.0)
(0, 78.0)
(0, 79.0)
(0, 80.0)
(0, 82.0)
(0, 84.0)
(0, 85.0)
(0, 86.0)
(0, 87.0)
(0, 88.0)
(0, 90.0)
(0, 93.0)
(0, 97.0)
(0, 102.0)
(0, 103.0)
(0, 106.0)
(0, 107.0)
(0, 109.0)
(0, 113.0)
(0, 114.0)
(0, 116.0)
(0, 117.0)
(0, 119.0)
(0, 121.0)
(0, 122.0)
(0, 124.0)
(0, 126.0)
(0, 129.0)
(0, 130.0)
(0, 131.0)
(1, 1.0)
(1, 3.0)
(1, 5.0)
(1, 6.0)
(1, 9.0)
(1, 12.0)
(1, 15.0)
(1, 16.0)
(1, 19.0)
(1, 20.0)
(1, 21.0)
(1, 30.0)
(1, 31.0)
(1, 38.0)
(1, 39.0)
(1, 41.0)
(1, 42.0)
(1, 43.0)
(1, 45.0)
(1, 46.0)
(1, 48.0)
(1, 49.0)
(1, 52.0)
(1,

(1, 91.0)
(1, 92.0)
(1, 95.0)
(1, 96.0)
(1, 98.0)
(1, 100.0)
(1, 101.0)
(1, 102.0)
(1, 103.0)
(1, 105.0)
(1, 106.0)
(1, 107.0)
(1, 109.0)
(1, 110.0)
(1, 111.0)
(1, 112.0)
(1, 115.0)
(1, 116.0)
(1, 117.0)
(1, 122.0)
(1, 124.0)
(1, 125.0)
(1, 127.0)
(1, 130.0)
(1, 131.0)
(1, 132.0)
(1, 136.0)
(1, 137.0)
(1, 138.0)
(1, 139.0)
(1, 140.0)
(1, 142.0)
(1, 150.0)
(1, 152.0)
(1, 156.0)
(1, 157.0)
(1, 161.0)
(1, 164.0)
(1, 165.0)
(1, 168.0)
(1, 172.0)
(1, 173.0)
(1, 176.0)
(1, 177.0)
(1, 179.0)
(1, 181.0)
(1, 182.0)
(1, 183.0)
(1, 185.0)
(1, 189.0)
(1, 194.0)
(1, 196.0)
(1, 197.0)
(1, 198.0)
(1, 199.0)
(1, 200.0)
(1, 202.0)
(1, 203.0)
(1, 204.0)
(1, 206.0)
(1, 207.0)
(1, 208.0)
(1, 210.0)
(1, 211.0)
(1, 212.0)
(1, 213.0)
(1, 215.0)
(1, 216.0)
(1, 218.0)
(1, 219.0)
(1, 223.0)
(1, 225.0)
(1, 228.0)
(1, 229.0)
(1, 230.0)
(1, 231.0)
(1, 234.0)
(1, 235.0)
(1, 237.0)
(1, 241.0)
(1, 243.0)
(1, 244.0)
(1, 246.0)
(1, 249.0)
(1, 252.0)
(1, 257.0)
(1, 261.0)
(1, 263.0)
(1, 264.0)
(1, 265.0)
(1, 266.0)
(1, 

In [61]:
import glob
training_pieces = []
for piece in glob.glob('../musicxml(notated)/*.mxl'):
    training_pieces.append(piece)
testing_pieces = []
for piece in glob.glob('../review/*.musicxml'):
    testing_pieces.append(piece)

In [62]:
print(training_pieces,testing_pieces)

['../musicxml(notated)\\Bach_Fugue_in_G_Minor_BWV_578_Piano_solo.mxl', '../musicxml(notated)\\Bach_Musette_in_D_BWV_Anh._126.mxl', '../musicxml(notated)\\C.P.E._Bach_Solfeggietto_in_C_minor.mxl', '../musicxml(notated)\\Chopin_-_Nocturne_Op_9_No_1_B_Flat_Minor.mxl', '../musicxml(notated)\\Etude_Opus_10_No._11__in_E_Major.mxl', '../musicxml(notated)\\E╠ütude_in_C_Minor.mxl', '../musicxml(notated)\\E╠ütude_in_F_Major.mxl', '../musicxml(notated)\\E╠ütude_in_F_Minor.mxl', '../musicxml(notated)\\E╠ütude_in_Gb_Major.mxl', '../musicxml(notated)\\E╠ütude_in_Gb_Major_Opus_25.mxl', '../musicxml(notated)\\G_Minor_Bach.mxl', '../musicxml(notated)\\Il_Vecchio_Castello.mxl', '../musicxml(notated)\\Menuet_in_G_Minor.mxl', '../musicxml(notated)\\Minuet_in_F.mxl', '../musicxml(notated)\\Minuet_in_G_Major_2nd.mxl', '../musicxml(notated)\\Moonlight_Sonata_1st_Movement.mxl', '../musicxml(notated)\\Nocturne_in_B_Major.mxl', '../musicxml(notated)\\Nocturne_in_C#_Minor.mxl', '../musicxml(notated)\\Nocturne_in

In [ ]:
from stable_baselines3 import DQN

env = SegmentationEnv(training_pieces)

model = DQN("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000, log_interval=4)
model.save("segmenatation_1")

env = SegmentationEnv(testing_pieces)
obs = env.reset()
while True:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    env.render()
    if done:
        break